## Hello!

This is an attempt to predict the winner of an NHL game given the NHL database given by Kaggle [here](https://www.kaggle.com/martinellis/nhl-game-data). After playing around with the data a lot, I found that home ice advantage played a role in the outcome of the game (home wins about 55% of the time). I looked into the head-to-head of each team vs. each other team and the results are found at this [imgur link](https://imgur.com/a/o316axx). After completing that, I wanted to find some feature that played a role in a team winning the game or not. Firstly, I mapped out where on the ice each team scored from the most, and where each team got scored on from the most - but this was not useful enough. Then, in this file, I checked the win rate if a team scores first - which was found to be useful and is explored below.

In [1]:
import pandas as pd
#Read in csv's.
#Need to replace team_id's because this database includes 33 teams (Phoenix and Atlanta had name changes to Arizona and Winnipeg)
#which the database defaulted to calling team_id 52,53,54. I replace them to have team_id's go from 1-31

iter_gp_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\game_plays.csv', iterator = True, chunksize = 1000)
game_plays_df = pd.concat([chunk[(chunk.event == 'Goal')][['game_id','team_id_for', 'team_id_against', 'event']] for chunk in iter_gp_csv])
game_plays_df = game_plays_df.replace([52,53,54], [11,27,31])

#Only read in Regular Season games. Again, replace necessary team id's
iter_game_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\game.csv', iterator = True, chunksize = 1000)
game_df = pd.concat([chunk[(chunk.type == 'R')] for chunk in iter_game_csv])
game_df = game_df.replace([52,53,54], [11,27,31])

iter_gts_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\game_teams_stats.csv', iterator = True, chunksize = 1000)
game_teams_stats_df = pd.concat([chunk for chunk in iter_gts_csv])
game_teams_stats_df = game_teams_stats_df.replace([52,53,54], [11,27,31])

iter_team_csv = pd.read_csv(r'C:\Users\SimoS\Documents\NHL\team_info.csv', iterator = True, chunksize = 1000)
team_df = pd.concat([chunk for chunk in iter_team_csv])

In [2]:
#Let's look at all the dataframes
game_plays_df.head(5)

,game_id,team_id_for,team_id_against,event
15,2011030221,1.0,4.0,Goal
133,2011030221,4.0,1.0,Goal
141,2011030221,4.0,1.0,Goal
161,2011030221,1.0,4.0,Goal
236,2011030221,4.0,1.0,Goal


In [3]:
game_df.head(5)

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
705,2013020674,20132014,R,2014-01-11,2014-01-11T03:00:00Z,19,23,1,2,home win REG,right,Rogers Arena,/api/v1/venues/null,America/Vancouver,-7,PDT
706,2013020177,20132014,R,2013-10-29,2013-10-29T23:00:00Z,24,4,3,2,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
707,2012020225,20122013,R,2013-02-19,2013-02-19T03:00:00Z,29,24,2,3,home win REG,left,Honda Center,/api/v1/venues/null,America/Los_Angeles,-7,PDT
708,2012020577,20122013,R,2013-04-08,2013-04-08T23:00:00Z,12,6,2,6,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
709,2013021085,20132014,R,2014-03-25,2014-03-25T23:30:00Z,17,29,2,4,home win REG,left,Nationwide Arena,/api/v1/venues/null,America/New_York,-4,EDT


In [4]:
game_teams_stats_df.head(5)

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways
0,2011030221,1,away,False,OT,Peter DeBoer,3,26,31,12,3,1,44.9,6,7
1,2011030221,4,home,True,OT,Peter Laviolette,4,36,27,6,6,1,55.1,13,4
2,2011030222,1,away,True,REG,Peter DeBoer,4,35,32,12,4,0,50.9,8,7
3,2011030222,4,home,False,REG,Peter Laviolette,1,20,24,32,5,0,49.1,9,6
4,2011030223,4,away,False,OT,Peter Laviolette,3,28,28,4,5,1,50.8,2,1


In [5]:
team_df.head(5)

,team_id,franchiseId,shortName,teamName,abbreviation,link
0,1,23,New Jersey,Devils,NJD,/api/v1/teams/1
1,4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
2,26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
3,14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
4,6,6,Boston,Bruins,BOS,/api/v1/teams/6


In [6]:
#These are some simple functions that will be used later in the file.

"""
Return a complete, cleaned teamList
"""
def teamList():
    teamList = team_df[['team_id','shortName','teamName']][(team_df.team_id != 11) & (team_df.team_id != 27)].sort_values("shortName")
    teamList = teamList.replace([52,53,54],[11,27,31])
    return teamList
"""
Get the team name given teamID
"""
def getTeamName(userTeamID):
    x = teamList()
    teamName = x[(x.team_id == userTeamID)]
    teamName = teamName[['teamName']]
    return teamName.iloc[0][0]



In [7]:
#Get the first goal of each game
goals_df = game_plays_df
goals_df = goals_df.drop_duplicates("game_id")
score_first_df = goals_df.merge(game_teams_stats_df, on='game_id')
score_first_df = score_first_df[score_first_df.team_id_for == score_first_df.team_id]
score_first_df.head(10)

,game_id,team_id_for,team_id_against,event,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways
0,2011030221,1.0,4.0,Goal,1,away,False,OT,Peter DeBoer,3,26,31,12,3,1,44.9,6,7
3,2011030222,4.0,1.0,Goal,4,home,False,REG,Peter Laviolette,1,20,24,32,5,0,49.1,9,6
4,2011030223,4.0,1.0,Goal,4,away,False,OT,Peter Laviolette,3,28,28,4,5,1,50.8,2,1
6,2011030224,4.0,1.0,Goal,4,away,False,REG,Peter Laviolette,2,22,23,10,2,1,62.5,20,5
9,2011030225,4.0,1.0,Goal,4,home,False,REG,Peter Laviolette,1,28,38,8,1,0,45.0,14,7
10,2011030411,26.0,1.0,Goal,26,away,True,OT,Darryl Sutter,2,25,35,4,1,0,55.4,6,4
12,2011030412,26.0,1.0,Goal,26,away,True,OT,Darryl Sutter,2,32,32,8,2,0,53.5,1,3
15,2011030413,26.0,1.0,Goal,26,home,True,REG,Darryl Sutter,4,21,55,12,2,2,56.6,17,5
16,2011030414,1.0,26.0,Goal,1,away,True,REG,Peter DeBoer,3,24,43,8,3,0,31.0,13,2
19,2011030415,1.0,26.0,Goal,1,home,True,REG,Peter DeBoer,2,19,34,6,2,1,52.8,4,3


In [8]:
#Get a cleaned team list
team_list = teamList()
team_df_modified = team_list[['team_id', 'shortName']]
team_df_modified

,team_id,shortName
17,24,Anaheim
31,27,Arizona
4,6,Boston
22,7,Buffalo
23,20,Calgary
30,12,Carolina
11,16,Chicago
24,21,Colorado
26,29,Columbus
25,25,Dallas


In [9]:
#See how many times each team scored first in the last 3 years (246 games). Merging with game_df eliminates playoff games.
score_first = pd.merge(score_first_df,game_df, on='game_id')[['game_id','team_id', 'HoA', 'won']]
score_first = score_first[((score_first.game_id.astype(str).str.startswith('2016'))
                            | (score_first.game_id.astype(str).str.startswith('2017')) 
                            | (score_first.game_id.astype(str).str.startswith('2018')))]
score_first.head(15)

,game_id,team_id,HoA,won
3125,2016020045,16,home,True
3126,2017020812,7,home,False
3129,2017020586,24,home,True
3130,2016020610,15,away,True
3132,2017020240,23,away,False
3133,2017020624,21,home,True
3135,2017020221,11,away,True
3136,2016020298,26,home,True
3137,2017020731,27,home,True
3138,2016020205,11,away,True


In [10]:
score_first_count = score_first['team_id'].value_counts().reset_index()
score_first_count = score_first_count.rename(columns={'team_id':'score_first','index':'team_id'})
score_first_count = score_first_count.merge(team_df_modified, on='team_id')
score_first_count 

,team_id,score_first,shortName
0,5,140,Pittsburgh
1,16,138,Chicago
2,19,137,St Louis
3,29,137,Columbus
4,15,136,Washington
5,10,135,Toronto
6,28,129,San Jose
7,27,128,Arizona
8,6,128,Boston
9,11,127,Winnipeg


In [11]:
home_wins_score_first = score_first[score_first.HoA == 'home']['won'].value_counts()
home_wins_score_first

True     1387
False     547
Name: won, dtype: int64

In [12]:
1387/(1387+547)

0.7171664943123061

Win 71.72% when scoring first at home. Let's look at scoring first at an away game

In [13]:
away_wins_score_first = score_first[score_first.HoA == 'away']['won'].value_counts()
away_wins_score_first


True     1120
False     674
Name: won, dtype: int64

In [14]:
1120/(1120+674)

0.6243032329988851

Win 62.24% when scoring first away. This is a big difference from the 70% at home. Home ice advantage is significant. Using what we now know (home ice advantage, scoring first, and how often a team scores first), let's see what training our model can do.

### This is currently being worked on and updated - this is just showing how to train a model

In [15]:
#Create train set (2016-2017, 2017-2018, 2018-2019 seasons)
train = game_df[['game_id', 'home_team_id', 'away_team_id', 'outcome']]
train = train[((train.game_id.astype(str).str.startswith('2016')) | (train.game_id.astype(str).str.startswith('2017')) | (train.game_id.astype(str).str.startswith('2018')))]

In [16]:
train.loc[train.outcome.str.contains('home'), 'outcome'] = 'home_win'
train.loc[train.outcome.str.contains('away',na=False), 'outcome'] = 'away_win'

In [17]:
train

,game_id,home_team_id,away_team_id,outcome
3885,2016020045,16,4,home_win
3886,2017020812,7,24,away_win
3889,2017020586,24,20,home_win
3890,2016020610,8,15,away_win
3892,2017020240,24,23,home_win
3893,2017020624,21,29,home_win
3895,2017020221,25,11,away_win
3896,2016020298,26,2,home_win
3897,2017020731,27,2,home_win
3898,2016020205,27,11,away_win


In [18]:
#Get head to head stats of the last 3 seasons
g = game_df[((game_df.type == "R") & ((game_df.game_id.astype(str).str.startswith('2016')) | (game_df.game_id.astype(str).str.startswith('2017')) 
                | (game_df.game_id.astype(str).str.startswith('2018'))))][['home_team_id', 'away_team_id', 'outcome']]

"""
This function is used to get the head_to_head stats of two teams.
"""
def train_head_to_head_aux(userTeam1, userTeam2):
    g_2 = g[(((g.home_team_id == userTeam2) &  (g.outcome.str.contains('home',na=False)))
             | ((g.away_team_id == userTeam2) & (g.outcome.str.contains('away',na=False))))]
    count = g_2['away_team_id'].value_counts()
    count2 = g_2['home_team_id'].value_counts()
    y = pd.concat([count, count2], axis = 1).reset_index()
    #Fix the NaN's
    y = y.fillna(0)
    userTeam2_name = getTeamName(userTeam2) + '_wins'
    y[userTeam2_name] = y['away_team_id'] + y['home_team_id']
    y[userTeam2_name] = pd.to_numeric(y[userTeam2_name], downcast='integer')
    y['away_team_id'] = pd.to_numeric(y['away_team_id'], downcast='integer')
    y = y.rename(columns={"away_team_id":"away_wins", "home_team_id":"home_wins","index":"team_id"})
    y = y[y.team_id == userTeam1]
    if y.empty == True:
        return 0
    return y.iloc[0][3]

### Need to populate training data with head to head stats:
Using our head_to_head function above, we can calculate a quotient in wins between each team. If Vegas has won 4 games against Chicago, and Chicago has won 2 against Vegas, it will give Vegas a 0.67 (4/6 = 2/3). 

In [19]:
home_team_list = train.home_team_id.to_list()
away_team_list = train.away_team_id.to_list()
count = 0
hth_list = []
for x in home_team_list:
    y = away_team_list[count]
    away = train_head_to_head_aux(y,x)
    home = train_head_to_head_aux(x, y)
    if away == 0:
        hth_list.append(1)
    else:
        hth_list.append(home / (home+away))
    count+=1


In [20]:
train['head_to_head'] = hth_list

In [21]:
train.head(10)

,game_id,home_team_id,away_team_id,outcome,head_to_head
3885,2016020045,16,4,home_win,0.666667
3886,2017020812,7,24,away_win,0.333333
3889,2017020586,24,20,home_win,0.384615
3890,2016020610,8,15,away_win,0.666667
3892,2017020240,24,23,home_win,0.307692
3893,2017020624,21,29,home_win,0.666667
3895,2017020221,25,11,away_win,0.692308
3896,2016020298,26,2,home_win,0.666667
3897,2017020731,27,2,home_win,0.666667
3898,2016020205,27,11,away_win,0.777778


In [22]:
#Merge with the scoring first rates we calculated near the beginning
train = train.merge(score_first_count, left_on='home_team_id', right_on='team_id')
train['home_score_first'] = train['score_first'] / 246
train = train.drop(columns=['team_id','score_first','shortName'])

In [23]:
train

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first
0,2016020045,16,4,home_win,0.666667,0.560976
1,2017021255,16,19,away_win,0.428571,0.560976
2,2016020178,16,25,home_win,0.384615,0.560976
3,2016020139,16,20,home_win,0.777778,0.560976
4,2016021067,16,21,home_win,0.500000,0.560976
5,2016020917,16,19,home_win,0.428571,0.560976
6,2017021040,16,12,away_win,0.666667,0.560976
7,2017020181,16,4,home_win,0.666667,0.560976
8,2016021152,16,29,home_win,0.500000,0.560976
9,2017020718,16,2,away_win,0.333333,0.560976


In [24]:
train = train.merge(score_first_count, left_on='away_team_id', right_on='team_id')
train

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first,team_id,score_first,shortName
0,2016020045,16,4,home_win,0.666667,0.560976,4,100,Philadelphia
1,2017020181,16,4,home_win,0.666667,0.560976,4,100,Philadelphia
2,2018021142,16,4,away_win,0.666667,0.560976,4,100,Philadelphia
3,2017020539,7,4,home_win,0.666667,0.471545,4,100,Philadelphia
4,2016020973,7,4,away_win,0.666667,0.471545,4,100,Philadelphia
5,2016020613,7,4,home_win,0.666667,0.471545,4,100,Philadelphia
6,2018020318,7,4,home_win,0.666667,0.471545,4,100,Philadelphia
7,2018020444,7,4,away_win,0.666667,0.471545,4,100,Philadelphia
8,2016020561,24,4,home_win,0.500000,0.500000,4,100,Philadelphia
9,2017020029,24,4,away_win,0.500000,0.500000,4,100,Philadelphia


In [25]:
train['away_score_first'] = train['score_first'] / 246
train

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first,team_id,score_first,shortName,away_score_first
0,2016020045,16,4,home_win,0.666667,0.560976,4,100,Philadelphia,0.406504
1,2017020181,16,4,home_win,0.666667,0.560976,4,100,Philadelphia,0.406504
2,2018021142,16,4,away_win,0.666667,0.560976,4,100,Philadelphia,0.406504
3,2017020539,7,4,home_win,0.666667,0.471545,4,100,Philadelphia,0.406504
4,2016020973,7,4,away_win,0.666667,0.471545,4,100,Philadelphia,0.406504
5,2016020613,7,4,home_win,0.666667,0.471545,4,100,Philadelphia,0.406504
6,2018020318,7,4,home_win,0.666667,0.471545,4,100,Philadelphia,0.406504
7,2018020444,7,4,away_win,0.666667,0.471545,4,100,Philadelphia,0.406504
8,2016020561,24,4,home_win,0.500000,0.500000,4,100,Philadelphia,0.406504
9,2017020029,24,4,away_win,0.500000,0.500000,4,100,Philadelphia,0.406504


In [26]:
train = train.drop(columns=['team_id','score_first','shortName'],axis=1)
train

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first,away_score_first
0,2016020045,16,4,home_win,0.666667,0.560976,0.406504
1,2017020181,16,4,home_win,0.666667,0.560976,0.406504
2,2018021142,16,4,away_win,0.666667,0.560976,0.406504
3,2017020539,7,4,home_win,0.666667,0.471545,0.406504
4,2016020973,7,4,away_win,0.666667,0.471545,0.406504
5,2016020613,7,4,home_win,0.666667,0.471545,0.406504
6,2018020318,7,4,home_win,0.666667,0.471545,0.406504
7,2018020444,7,4,away_win,0.666667,0.471545,0.406504
8,2016020561,24,4,home_win,0.500000,0.500000,0.406504
9,2017020029,24,4,away_win,0.500000,0.500000,0.406504


In [27]:
train['%_score_first'] = (train['home_score_first'] / (train['away_score_first']+ train['home_score_first']))
train

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first,away_score_first,%_score_first
0,2016020045,16,4,home_win,0.666667,0.560976,0.406504,0.579832
1,2017020181,16,4,home_win,0.666667,0.560976,0.406504,0.579832
2,2018021142,16,4,away_win,0.666667,0.560976,0.406504,0.579832
3,2017020539,7,4,home_win,0.666667,0.471545,0.406504,0.537037
4,2016020973,7,4,away_win,0.666667,0.471545,0.406504,0.537037
5,2016020613,7,4,home_win,0.666667,0.471545,0.406504,0.537037
6,2018020318,7,4,home_win,0.666667,0.471545,0.406504,0.537037
7,2018020444,7,4,away_win,0.666667,0.471545,0.406504,0.537037
8,2016020561,24,4,home_win,0.500000,0.500000,0.406504,0.551570
9,2017020029,24,4,away_win,0.500000,0.500000,0.406504,0.551570


In [28]:

train['head_to_head'] = train['head_to_head']*100

In [61]:
train['%_score_first'] = train['%_score_first']

In [62]:
train

,game_id,home_team_id,away_team_id,outcome,head_to_head,home_score_first,away_score_first,%_score_first
0,2016020045,16,4,home_win,66.666667,0.560976,0.406504,0.579832
1,2017020181,16,4,home_win,66.666667,0.560976,0.406504,0.579832
2,2018021142,16,4,away_win,66.666667,0.560976,0.406504,0.579832
3,2017020539,7,4,home_win,66.666667,0.471545,0.406504,0.537037
4,2016020973,7,4,away_win,66.666667,0.471545,0.406504,0.537037
5,2016020613,7,4,home_win,66.666667,0.471545,0.406504,0.537037
6,2018020318,7,4,home_win,66.666667,0.471545,0.406504,0.537037
7,2018020444,7,4,away_win,66.666667,0.471545,0.406504,0.537037
8,2016020561,24,4,home_win,50.000000,0.500000,0.406504,0.551570
9,2017020029,24,4,away_win,50.000000,0.500000,0.406504,0.551570


In [63]:
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm

In [64]:

train_1, test = train_test_split(train, test_size = 0.3)# in this our main data is split into train


In [65]:
train_X = train_1[['home_team_id','away_team_id','head_to_head','%_score_first']]
train_y = train_1.outcome

In [66]:
test_X=test[['home_team_id','away_team_id','head_to_head','%_score_first']]

In [67]:
test_y = test.outcome

In [68]:
train_X.head(5)

,home_team_id,away_team_id,head_to_head,%_score_first
2877,2,1,33.333333,0.500000
196,28,19,55.555556,0.484962
34,1,4,38.461538,0.555556
2930,19,1,0.000000,0.522901
3245,12,27,50.000000,0.490040


In [69]:
test_X.head(5)

,home_team_id,away_team_id,head_to_head,%_score_first
180,30,19,38.461538,0.454183
3456,8,5,66.666667,0.469697
985,16,30,38.461538,0.547619
2947,22,1,33.333333,0.481328
3315,22,27,38.461538,0.475410


In [70]:
train_y.head(5)

2877    home_win
196     away_win
34      away_win
2930    home_win
3245    away_win
Name: outcome, dtype: object

In [71]:
model = svm.SVC() #select the algorithm
model.fit(train_X,train_y) # we train the algorithm with the training data and the training output
prediction=model.predict(test_X) #now we pass the testing data to the trained algorithm
print('The accuracy of the SVM is:',metrics.accuracy_score(prediction,test_y))#now we check the accuracy of the algorithm. 
#we pass the predicted output by the model and the actual output

c:\users\simos\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


The accuracy of the SVM is: 0.5556537102473498


In [72]:
#Logistic regression
model = LogisticRegression()
model.fit(train_X,train_y)
prediction=model.predict(test_X)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction,test_y))

The accuracy of the Logistic Regression is 0.6713780918727915


c:\users\simos\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [73]:
#Decision Tree
model=DecisionTreeClassifier()
model.fit(train_X,train_y)
prediction=model.predict(test_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction,test_y))

The accuracy of the Decision Tree is 0.5353356890459364


In [74]:
#KNN
model=KNeighborsClassifier(n_neighbors=3) #this examines 3 neighbours for putting the new data into a class
model.fit(train_X,train_y)
prediction=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction,test_y))

The accuracy of the KNN is 0.5556537102473498


In [60]:
#Therefore logistic regression is the best

### Any feedback is really appreciated. I am just dipping my toes in the water.